In [1]:
!pip install transformers



[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from transformers import AutoTokenizer, AutoModel
from transformers import AutoTokenizer, AutoModelForTokenClassification

import pandas as pd
import torch

c:\Users\ruebe\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import pandas as pd

# Load the CSV file
df = pd.read_csv("podcast_transcriptions.csv")

# Display the first 5 rows
df.head()


,episode_name,duration_seconds,transcription
0,Go ikatisa morago ga go bona moento wa COVID19,498.98,eketshedimosetso ya kowid oitlisediwa ke mots...
1,Spunik not being accepted by SAHPRA,419.91,e ketshedimosetso ya kowid oitlisediwa ke mot...
2,Pegelo ya DIGITAL VIBES e bontshang ka mo madi...,436.62,e ketshedimosetso ya kowid naoitlisediwa ke mo...
3,School children_s mental health crisis during ...,559.32,e kitshedimosetso ya kowid noitlisediwa ke mot...
4,Go rwesa batho baba sa entelwang COVID-19 mola...,423.75,e kekgatiso ya motswedinfm konka mo kamooso eg...


In [4]:
# View column names
print(df.columns)

# Get basic info
df.info()

# Display episode names and durations
df[["episode_name", "transcription"]].head(10)


Index(['episode_name', 'duration_seconds', 'transcription'], dtype='object')
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59 entries, 0 to 58
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   episode_name      59 non-null     object 
 1   duration_seconds  59 non-null     float64
 2   transcription     59 non-null     object 
dtypes: float64(1), object(2)
memory usage: 1.5+ KB


,episode_name,transcription
0,Go ikatisa morago ga go bona moento wa COVID19,eketshedimosetso ya kowid oitlisediwa ke mots...
1,Spunik not being accepted by SAHPRA,e ketshedimosetso ya kowid oitlisediwa ke mot...
2,Pegelo ya DIGITAL VIBES e bontshang ka mo madi...,e ketshedimosetso ya kowid naoitlisediwa ke mo...
3,School children_s mental health crisis during ...,e kitshedimosetso ya kowid noitlisediwa ke mot...
4,Go rwesa batho baba sa entelwang COVID-19 mola...,e kekgatiso ya motswedinfm konka mo kamooso eg...
5,C1 2 Variant,e ketshedimosetso ya kofid nuitlisediwake mo ...
6,WHO says Omicron is dangerous for the unvaccin...,e kitshedimosetso ya kowid nauitlisediwake mot...
7,Moento o ama jang boMme ba ba anyisang,e ketshedimosetso ya kowid n oitlisediwa ke mo...
8,Ditsela tse di maleba tsa go buisana le go pha...,eketshedimosetso ya kowid oitlisediwa ke mo ts...
9,Booster basics-health workers will get booster...,e ketshedimosetso ya kowid noitlisediwa ke mot...


In [5]:
tokenizer = AutoTokenizer.from_pretrained("dsfsi/PuoBERTa-POS")
model = AutoModelForTokenClassification.from_pretrained("dsfsi/PuoBERTa-POS")

In [ ]:
def split_sentences(text, max_length=30):
    tokens = tokenizer.tokenize(text)
    sentences = []
    current = []

    for token in tokens:
        current.append(token)
        if len(current) >= max_length or token in ['ke', 'mme', 'fa', 'jalo', 'le', 'ya', 'a', 'go', 'e']:
            sentences.append(tokenizer.convert_tokens_to_string(current).replace(" ##", "").strip())
            current = []

    if current:
        sentences.append(tokenizer.convert_tokens_to_string(current).replace(" ##", "").strip())

    return sentences


In [7]:
# Example transcription text
raw_text = """
eketshedimosetso ya kowid  oitlisediwa ke motswedinya fem konka bokamosoeeponso ...
"""  # Truncated for brevity — paste your full text here


In [8]:
split = split_sentences(raw_text)

# Display result
for i, sentence in enumerate(split, 1):
    print(f"{i}. {sentence}")


1. eketshedimosetso ya kowid  oitlisediwa ke motswedinya fem konka bokamosoe
2. eponso ...


In [9]:
def process_transcriptions(df):
    processed_transcriptions = []
    for transcription in df['transcription']:
        sentences = split_sentences(transcription)
        numbered_sentences = [f"{i}. {sentence}" for i, sentence in enumerate(sentences, 1)]
        processed_transcriptions.append("\n".join(numbered_sentences))
    df['transcription'] = processed_transcriptions

process_transcriptions(df)

In [10]:
df.to_csv("podcast_transcriptions_numberedSentences.csv", index=False)

In [11]:
for index, row in df.iterrows():
    print(f"\nEpisode: {row['episode_name']}\n")
    print(f"Transcription:\n{row['transcription']}")
    print("-" * 80)


Episode: Go ikatisa morago ga go bona moento wa COVID19

Transcription:
1. eketshedimosetso ya kowid  oitlisediwa ke motswedinya fem konka bokamosoe
2. eponso a koruke
3. moretsiwa botsafa
4. komofa
5. rtatlenke a bona potsweki ya moentolee go tswalelapele ka botshelojwa gago fa o fetsa go tlhaba a motho wa kgona go dira sa
6. tlwasingo s dira jaka go ikad aka gongwefela famothoa
7. fetsa go thabaa
8. emon a ka ritgoremu ento o na le ditlamorago tetsetsetse malwa pe dilenpeng okaupua ke tlhogo oka ukwe
9. karemnele o botlhoko oka  oka uka botlhupgape fo o tlhadilenpeng tetey lo yal okaupu a tsa
10. tidilokau twatiphogo
11. ki binki ditshipo kapa ditamorago te ditsa moento mme baitan e sa ga baiti gore ke pefeng tee o tlo dibonang weena
12. mmongwe le mongwe a tlo dibonang morago ga a entiwa nne geraiti gore e ga ukuta otutoto nasentla mo mmelen
13. topatapa ukuta ekare o lapila uta pone e kare o na le ntlhikela o ferogadibete o pue
14. kitlhogo kapamme
15. lwago
16. e kare o na le  bo